In [1]:
import pandas as pd
from preProccesPuzzle import PreProcess
from clue_classifier import ClueClassifier
from constraints import Constraint, IdentityConstrain, NextToConstrain, DistanceConstrain,RightConstrain, LeftConstrain, DirectRightConstrain, DirectLeftConstrain, PositionAbsoluteConstrain, PositionAbsoluteNegativeConstrain

In [2]:
gridmode = pd.read_parquet("Gridmode-00000-of-00001.parquet")
mc = pd.read_parquet("mc-00000-of-00001.parquet")
ppp = PreProcess()
classifier = ClueClassifier()

In [ ]:
puzzle = gridmode.puzzle.iloc[2].lower()

attrs, clues = ppp.proccess(puzzle)

print(puzzle)

print(attrs.keys())
for k in attrs.keys():
    print(attrs[k])
print()

for c in clues:
    clue, clue_type = classifier.classify(c)

    constr = None
    if clue_type == "IDENTITY":
        constr = IdentityConstrain(attrs, clue)
    if clue_type == "NEXT_TO":
        constr = NextToConstrain(attrs, clue)
    if clue_type == "LEFT":
        constr = LeftConstrain(attrs, clue)
    if clue_type == "RIGHT":
        constr = RightConstrain(attrs, clue)
    if clue_type == "DISTANCE":
        constr = DistanceConstrain(attrs, clue)
    if clue_type == "DIRECT_LEFT":
        constr = DirectLeftConstrain(attrs, clue)
    if clue_type == "DIRECT_RIGHT":
        constr = DirectRightConstrain(attrs, clue)
    if clue_type == "POSITION_ABSOLUTE":
        constr = PositionAbsoluteConstrain(attrs, clue)
    if clue_type == "POSITION_ABSOLUTE_NEGATIVE":
        constr = PositionAbsoluteNegativeConstrain(attrs, clue)
        

    if constr:
        print(constr.get_info())




there are 6 houses, numbered 1 to 6 from left to right, as seen from across the street. each house is occupied by a different person. each house has a unique attribute for each of the following characteristics:
 - each person has a unique name: `bob`, `alice`, `peter`, `eric`, `arnold`, `carol`
 - each mother is accompanied by their child: `fred`, `timothy`, `samantha`, `alice`, `meredith`, `bella`
 - people have unique favorite music genres: `pop`, `hip hop`, `classical`, `jazz`, `rock`, `country`
 - people have unique heights: `average`, `very tall`, `tall`, `super tall`, `very short`, `short`

## clues:
1. there is one house between the person's child is named samantha and the person who is short.
2. the person's child is named alice is bob.
3. the person who loves country music is directly left of arnold.
4. alice is the person who is tall.
5. the person who loves pop music is eric.
6. bob is somewhere to the right of the person who is super tall.
7. the person's child is named fre